In [1]:
from io import StringIO

from pandas import read_csv

from swmm_api import SwmmInput
from swmm_api.input_file import section_labels as sections
from swmm_api.input_file.section_types import SECTION_TYPES

from swmm_api.input_file.sections import Timeseries
from swmm_api.input_file.sections.others import TimeseriesData, TimeseriesFile

In [2]:
csv_file = StringIO(""",TS1
0:05,1.7
0:10,2.3
0:15,3.4
0:20,8
0:25,1.15 """)


In [3]:
series = read_csv(csv_file, index_col=0, squeeze=True)
series

0:05    1.70
0:10    2.30
0:15    3.40
0:20    8.00
0:25    1.15
Name: TS1, dtype: float64

In [4]:
inp = SwmmInput()
inp

{}

In [5]:
# inp[sections.TIMESERIES] = Timeseries.create_section()
inp[sections.TIMESERIES] = SECTION_TYPES[sections.TIMESERIES].create_section()
print(inp[sections.TIMESERIES].to_inp_lines())

;; No Data


In [6]:
inp[sections.TIMESERIES].add_obj(TimeseriesData('TS1', data=list(zip(series.index,series))))
print(inp[sections.TIMESERIES].to_inp_lines())


TS1 00:05 1.7
TS1 00:10 2.3
TS1 00:15 3.4
TS1 00:20 8.0
TS1 00:25 1.15



In [7]:
inp[sections.TIMESERIES]['TS1']

TimeseriesData(Name = "TS1", data = [(0.08333333333333333, 1.7), (0.16666666666666666, 2.3), (0.25, 3.4), (0.3333333333333333, 8.0), (0.4166666666666667, 1.15)])

In [8]:
inp[sections.TIMESERIES]['TS1'].frame

0.083333    1.70
0.166667    2.30
0.250000    3.40
0.333333    8.00
0.416667    1.15
Name: TS1, dtype: float64

Wenn die Zeitserie einen Namen hat und der Index bereits das richtige Format für SWMM besitzt,
kann das Object so erstellt werden.

In [9]:
series.name='TS3'
inp[sections.TIMESERIES].add_obj(TimeseriesData.from_pandas(series))
print(inp[sections.TIMESERIES].to_inp_lines())


TS1 00:05 1.7
TS1 00:10 2.3
TS1 00:15 3.4
TS1 00:20 8.0
TS1 00:25 1.15

TS3 00:05 1.7
TS3 00:10 2.3
TS3 00:15 3.4
TS3 00:20 8.0
TS3 00:25 1.15



In [10]:
inp[sections.TIMESERIES].add_obj(TimeseriesFile('TS2', filename='example_file.dat'))
print(inp[sections.TIMESERIES].to_inp_lines())


TS1 00:05 1.7
TS1 00:10 2.3
TS1 00:15 3.4
TS1 00:20 8.0
TS1 00:25 1.15

TS2 FILE "example_file.dat"
TS3 00:05 1.7
TS3 00:10 2.3
TS3 00:15 3.4
TS3 00:20 8.0
TS3 00:25 1.15



In [11]:
inp[sections.TIMESERIES]['TS2']

TimeseriesFile(Name = "TS2", kind = "FILE", filename = "example_file.dat")

In [12]:
rain = """duration,1,2,3,5,10,20,30,50,100
5,5.4,7,8,9.3,10.9,12.6,13.6,14.8,16.5
10,8.4,10.7,12.1,13.9,16.2,18.6,20,21.7,24.1
15,10.3,13.2,14.9,17,19.9,22.8,24.5,26.6,29.5
20,11.6,15,16.9,19.4,22.7,26,28,30.4,33.8
30,13.4,17.4,19.8,22.8,26.9,31,33.4,36.4,40.4
45,14.8,19.8,22.7,26.4,31.4,36.4,39.3,43,48
60,15.7,21.4,24.8,29.1,34.8,40.5,43.9,48.2,53.9
90,17,22.8,26.2,30.5,36.3,42.1,45.5,49.8,55.6
120,17.9,23.8,27.2,31.6,37.4,43.3,46.7,51,56.9
180,19.4,25.3,28.8,33.2,39.1,45,48.5,52.9,58.8
240,20.5,26.5,30,34.4,40.4,46.3,49.8,54.2,60.2
360,22.2,28.2,31.8,36.2,42.3,48.3,51.8,56.3,62.3
540,24,30.1,33.7,38.2,44.3,50.4,54,58.5,64.6
720,25.4,31.5,35.1,39.7,45.8,52,55.6,60.1,66.3
1080,27.4,33.7,37.3,41.9,48.1,54.3,58,62.6,68.8
1440,29,35.3,38.9,43.6,49.9,56.1,59.8,64.4,70.7
2880,36.2,44.6,49.6,55.8,64.2,72.7,77.6,83.8,92.3
4320,41.2,50.9,56.6,63.7,73.5,83.2,88.8,96,105.7
"""
import io
from pandas import read_csv
from ehyd_tools.synthetic_rainseries import RainModeller
kostra = read_csv(io.StringIO(rain), index_col=0)
model_rain = RainModeller()
model_rain.idf_table = kostra
model_rain.idf_table.columns = model_rain.idf_table.columns.astype(int)
ts = model_rain.euler.get_time_series(return_period=3, duration=60, interval=5, kind=2, start_time='2021-01-01 00:00')
ts = ts.round(2)
ts = ts.rename('KOSTRA')
ts

2021-01-01 00:00:00    0.00
2021-01-01 00:05:00    2.00
2021-01-01 00:10:00    2.80
2021-01-01 00:15:00    4.10
2021-01-01 00:20:00    8.00
2021-01-01 00:25:00    1.45
2021-01-01 00:30:00    1.45
2021-01-01 00:35:00    0.97
2021-01-01 00:40:00    0.97
2021-01-01 00:45:00    0.97
2021-01-01 00:50:00    0.70
2021-01-01 00:55:00    0.70
2021-01-01 01:00:00    0.70
Name: KOSTRA, dtype: float64

In [13]:
from swmm_api.input_file.sections.others import TimeseriesData


In [15]:
TimeseriesData.from_pandas(ts)

TimeseriesData(Name = "KOSTRA", data = [(Timestamp('2021-01-01 00:00:00'), 0.0), (Timestamp('2021-01-01 00:05:00'), 2.0), (Timestamp('2021-01-01 00:10:00'), 2.8), (Timestamp('2021-01-01 00:15:00'), 4.1), (Timestamp('2021-01-01 00:20:00'), 8.0), (Timestamp('2021-01-01 00:25:00'), 1.45), (Timestamp('2021-01-01 00:30:00'), 1.45), (Timestamp('2021-01-01 00:35:00'), 0.97), (Timestamp('2021-01-01 00:40:00'), 0.97), (Timestamp('2021-01-01 00:45:00'), 0.97), (Timestamp('2021-01-01 00:50:00'), 0.7), (Timestamp('2021-01-01 00:55:00'), 0.7), (Timestamp('2021-01-01 01:00:00'), 0.7)])